# NHANES dataset: exploratory and analysis
> This notebook has the purpose to analyse NHANES dataset for future analysis 

## Install nhanesA packages

In [2]:
install.packages("nhanesA")


The downloaded binary packages are in
	/var/folders/8d/09j2zdrx7klfl7nffqz630000000gn/T//RtmpZO70qN/downloaded_packages


In [3]:
install.packages("devtools")
devtools::install_github("cjendres1/nhanes")


The downloaded binary packages are in
	/var/folders/8d/09j2zdrx7klfl7nffqz630000000gn/T//RtmpZO70qN/downloaded_packages


Using GitHub PAT from the git credential store.




rlang (1.1.4 -> 1.1.5) [CRAN]


Installing 1 packages: rlang




The downloaded binary packages are in
	/var/folders/8d/09j2zdrx7klfl7nffqz630000000gn/T//RtmpZO70qN/downloaded_packages
-- R CMD build -----------------------------------------------------------------
* checking for file '/private/var/folders/8d/09j2zdrx7klfl7nffqz630000000gn/T/RtmpZO70qN/remotes10ba27e70a690/cjendres1-nhanes-d98e5e6/DESCRIPTION' ... OK
* preparing 'nhanesA':
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building 'nhanesA_1.3.tar.gz'



## Browse Nhanes website

In [ ]:
nhanesA::browseNHANES(
    year = 2023,
    data_group = 'LAB'
)

## Create dataframe using NHANES codes

In [ ]:
DF <- nhanesA::nhanes(nh_table = 'OHQ')
DF$OHQ080

[1] <NA>       No         <NA>       No         <NA>       <NA>      
   [7] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
  [13] No         <NA>       No         <NA>       <NA>       <NA>      
  [19] <NA>       <NA>       <NA>       Yes        <NA>       <NA>      
  [25] <NA>       <NA>       <NA>       <NA>       <NA>       No        
  [31] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
  [37] <NA>       <NA>       <NA>       <NA>       No         <NA>      
  [43] <NA>       <NA>       <NA>       <NA>       No         <NA>      
  [49] <NA>       <NA>       No         <NA>       <NA>       <NA>      
  [55] No         <NA>       <NA>       <NA>       <NA>       <NA>      
  [61] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
  [67] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
  [73] <NA>       <NA>       <NA>       <NA>       No         <NA>      
  [79] <NA>       <NA>       No         <NA>       <NA>       No        
  [85] No         No         No         <NA>       <NA>       <NA>      
  [91] No         No         <NA>       <NA>       <NA>       <NA>      
  [97] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
 [103] <NA>       <NA>       No         <NA>       <NA>       No        
 [109] <NA>       <NA>       No         No         <NA>       <NA>      
 [115] <NA>       <NA>       <NA>       No         No         No        
 [121] Yes        No         <NA>       <NA>       No         <NA>      
 [127] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
 [133] <NA>       <NA>       <NA>       <NA>       <NA>       No        
 [139] <NA>       <NA>       No         <NA>       <NA>       <NA>      
 [145] <NA>       <NA>       <NA>       No         <NA>       <NA>      
 [151] No         No         <NA>       <NA>       <NA>       No        
 [157] <NA>       <NA>       <NA>       <NA>       No         <NA>      
 [163] <NA>       No         No         No         <NA>       <NA>      
 [169] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
 [175] <NA>       No         No         <NA>       <NA>       <NA>      
 [181] No         <NA>       <NA>       No         <NA>       <NA>      
 [187] <NA>       <NA>       <NA>       <NA>       <NA>       No        
 [193] No         <NA>       <NA>       <NA>       No         <NA>      
 [199] <NA>       <NA>       <NA>       No         <NA>       <NA>      
 [205] <NA>       <NA>       <NA>       <NA>       No         <NA>      
 [211] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
 [217] <NA>       No         <NA>       <NA>       <NA>       <NA>      
 [223] No         No         <NA>       <NA>       <NA>       <NA>      
 [229] <NA>       <NA>       <NA>       <NA>       No         <NA>      
 [235] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
 [241] <NA>       No         <NA>       No         <NA>       <NA>      
 [247] <NA>       No         <NA>       <NA>       <NA>       <NA>      
 [253] <NA>       No         No         <NA>       No         <NA>      
 [259] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
 [265] <NA>       <NA>       <NA>       <NA>       No         <NA>      
 [271] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
 [277] <NA>       Yes        <NA>       <NA>       <NA>       <NA>      
 [283] <NA>       <NA>       <NA>       No         <NA>       <NA>      
 [289] <NA>       <NA>       No         <NA>       <NA>       <NA>      
 [295] No         <NA>       <NA>       <NA>       <NA>       <NA>      
 [301] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
 [307] <NA>       No         <NA>       No         No         No        
 [313] <NA>       <NA>       <NA>       <NA>       No         <NA>      
 [319] <NA>       <NA>       <NA>       <NA>       <NA>       <NA>      
 [325] <NA>       <NA>       <NA>       No         No        

In [22]:
nhanesA::nhanesCodebook('OHQ', 'OHQ080')

Code or Value,Value Description,Count,Cumulative,Skip to Item
<chr>,<chr>,<int>,<int>,<lgl>
1,Yes,147,147,NA
2,No,1718,1865,NA
7,Refused,0,1865,NA
9,Don't know,1,1866,NA
.,Missing,7364,9230,NA


In [19]:
head(DF[ , c("OHQ020", "OHQ040", "OHQ050", "OHQ080", "OHQ100", "OHQ110")])

,OHQ020,OHQ040,OHQ050,OHQ080,OHQ100,OHQ110
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,NA,NA,NA,NA,NA,NA
2,Never?,NA,2 or more times a year,No,No,No
3,NA,NA,Once a year,NA,NA,NA
4,Never?,Yes,2 or more times a year,No,No,No
5,Never?,NA,Once a year,NA,NA,NA
6,Never?,NA,NA,NA,NA,NA


### How to drop all NA values

In [20]:
DF_filtered <- na.omit(DF[ , c("OHQ020", "OHQ040", "OHQ050", "OHQ080", "OHQ100", "OHQ110")])
dim(DF_filtered)

[1] 397   6

## Explore Data Groups such as Questionnarie, Demographics...

In [17]:
nhanesA::nhanesTables(data_group = 'QUESTIONNAIRE', year = 2000)

Data.File.Name,Data.File.Description
<chr>,<chr>
RXQ_RX,Prescription Medications
CIQGAD,Mental Health - Generalized Anxiety Disorder
CIQMDEP,Mental Health - Depression
CIQPANIC,Mental Health - Panic Disorder
ACQ,Acculturation
ALQ,Alcohol Use
AUQ,Audiometry
BAQ,Balance
BPQ,Blood Pressure & Cholesterol


In [16]:
exam = nhanesA::nhanesTables('QUESTIONNAIRE', 2000)
dim(exam)

[1] 43  2

In [11]:
nhanesA::nhanesTableSummary('DEMO_D', use = "codebook")

table,varname,label,nobs_cb,na_cb,has_range,nlevels,skip
<chr>,<chr>,<chr>,<int>,<int>,<lgl>,<int>,<lgl>
DEMO_D,SEQN,Respondent sequence number,NA,NA,NA,NA,NA
DEMO_D,SDDSRVYR,Data Release Number,10348,0,FALSE,2,FALSE
DEMO_D,RIDSTATR,Interview/Examination Status,10348,0,FALSE,3,FALSE
DEMO_D,RIDEXMON,Six month time period,10348,398,FALSE,3,FALSE
DEMO_D,RIAGENDR,Gender,10348,0,FALSE,3,FALSE
DEMO_D,RIDAGEYR,Age at Screening Adjudicated - Recode,10348,0,TRUE,3,FALSE
DEMO_D,RIDAGEMN,Age in Months - Recode,10348,170,TRUE,2,FALSE
DEMO_D,RIDAGEEX,Exam Age in Months - Recode,10348,571,TRUE,2,FALSE
DEMO_D,RIDRETH1,Race/Ethnicity - Recode,10348,0,FALSE,6,FALSE
